### Modelli teorici utilizzati:
* **viscosità**: ... (indicato dal prof)
* **densità**: [Density model for aqueous glycerol solutions](https://link.springer.com/article/10.1007/s00348-018-2527-y) (trovato da noi)

In [1]:
from rberga06.phylab import Datum as d, Measure
import math
import sympy
from typing import Any, Iterable
from itertools import count
sympy.init_printing()

def abs_(x: Measure[float]) -> Measure[float]:
    return d(abs(x.best), x.delta)

def sin(x: Measure[float]) -> Measure[float]:
    return d(math.sin(x.best), abs(math.cos(x.best)) * x.delta)

def exp(x: Measure[float]) -> Measure[float]:
    e_x = math.exp(x.best)
    return d(e_x, e_x * x.delta)

def ln(x: Measure[float]) -> Measure[float]:
    return d(math.log(x.best), x.delta_rel)

def expr_delta(expr: Any, symbols: Iterable[sympy.Symbol], /) -> Any:
    return sympy.sqrt(sum([(expr.diff(x) * sympy.Symbol(f"(\\delta {x})"))**2 for x in symbols], start=sympy.Integer(0)))

def evalf(expr: Any, subs: dict[sympy.Symbol, Measure[float]], /, *, delta: Any = None) -> Measure[float]:
    best_subs = {x: X.best for x, X in subs.items()}
    delta_subs = {sympy.Symbol(f"(\\delta {x})"): X.delta for x, X in subs.items()}
    if delta is None:
        delta = expr_delta(expr, subs)
    return d(expr.evalf(subs=best_subs), delta.evalf(subs=best_subs | delta_subs))

$$\alpha = 1 - x + \frac{abx(1-x)}{ax+b(1-x)}$$
$$(ax+b(1-x))\alpha = (ax+b(1-x))(1-x) + abx(1-x)$$
$$a\alpha x+b\alpha(1-x) = ax(1-x) + b(1-x)^2 + abx(1-x)$$
$$a\alpha x+b\alpha-b\alpha x = ax-ax^2 + b+bx^2-2bx + abx-abx^2$$
$$(a - b + ab) x^2 + (a\alpha-b\alpha - a + 2b - ab) x + b\alpha - b = 0$$

In [2]:
def eta_water(T: Measure[float], /) -> Measure[float]:  # mPa s
    _T = sympy.symbols("T")
    e = 1.79 * sympy.exp((-1230 - _T)*_T/(36100 + 360 * _T))  # type: ignore
    return evalf(e, {_T: T})

def eta_glycerol(T: Measure[float], /) -> Measure[float]:   # mPa s
    _T = sympy.symbols("T")
    e = 12100 * sympy.exp((-1233 - _T)*_T/(9900 + 70 * _T))  # type: ignore
    return evalf(e, {_T: T})

def rho_water(T: Measure[float], /) -> Measure[float]:   # kg / m^3
    return 1000 * (1 - abs_((T - 3.98)/615)**1.71)

def rho_glycerol(T: Measure[float], /) -> Measure[float]:   # kg/m^3
    return (1273 - 0.612 * T)

def _kappa_solution(T: Measure[float], w_g: Measure[float], /) -> Measure[float]:
    A = (1.78e-6 * T**2 - 1.82e-4 * T + 1.41e-2)
    return 1 + A * sin(w_g ** 1.31 * math.pi) ** 0.81

def rho_solution(T: Measure[float], w_g: Measure[float], /) -> Measure[float]:  # g/dm^3
    rho_w = rho_water(T)
    rho_g = rho_glycerol(T)
    # w_g: concentrazione massa/massa
    return _kappa_solution(T, w_g) * (rho_w + (rho_g - rho_w)/(1 + (rho_g/rho_w) * (1/w_g - 1)))


def _solve_alpha2wg() -> tuple[Any, Any, Any, Any]:
    T, alpha, x, a, b = sympy.symbols("T \\alpha x a b")
    equation = (1 - x) + (a * b * x * (1 - x))/(a * x + b * (1 - x)) - alpha
    x1, x2 = sympy.solve(equation, x)
    a_expr = 0.705 - 0.0017 * T
    b_expr = ((4.9 + 0.036 * T) * a ** 2.5).subs({a: a_expr})
    x1 = x1.subs({a: a_expr, b: b_expr})
    x2 = x2.subs({a: a_expr, b: b_expr})
    return x1, expr_delta(x1, [T, alpha]), x2, expr_delta(x2, [T, alpha])
_SOLUTIONS_ALPHA2WG = _solve_alpha2wg()

def alpha2wg(T: Measure[float], alpha: Measure[float]) -> Measure[float]:
    _T, _alpha = sympy.symbols("T \\alpha")
    _x1, _x1delta, _x2, _x2delta = _SOLUTIONS_ALPHA2WG
    x1 = evalf(_x1, {_T: T, _alpha: alpha}, delta=_x1delta)
    x2 = evalf(_x2, {_T: T, _alpha: alpha}, delta=_x2delta)
    # old code
    if False:
        a = 0.705 - 0.0017 * T
        b = (4.9+0.036 * T) * a**2.5
        A = (a - b + a*b)
        B = (a * alpha - b * alpha - a + 2*b - a*b)
        Delta = B**2 - 4*A*(b*alpha - b)
        x1 = (-B + Delta ** .5)/(2*A)
        x2 = (-B - Delta ** .5)/(2*A)
    return min(x1, x2, key=lambda x: x.best)


$$v=\frac{2}{9}(\rho_s - \rho_f) \frac{R^2}{\eta} g$$
$$R^2 = \xi v$$
$$R^2 = \frac{1}{\frac{2}{9}(\rho_s - \rho_f) \frac{g}{\eta}} v$$
$$\xi = \frac{9\eta}{2(\rho_s - \rho_f) g}$$
$$\frac{2(\rho_s - \rho_f) g \xi}{9} = \eta$$

In [3]:
g = d(9.806, 0.01)  # m/s^2
rho_sfere = d(7713, 45)  # kg/m^3

def analysis(T: Measure[float], xi: Measure[float], best_threshold: float = 1e-3) -> tuple[Measure[float], Measure[float], Measure[float]]:
    print("T =", T, "°C")
    print("ξ =", xi)
    print("η(acqua) =", _eta_water := eta_water(T), "mPa s")
    print("η(glicerina) =", _eta_glycerol := eta_glycerol(T), "mPa s")
    print("⍴(acqua) =", rho_water(T), "kg/m³")
    print("⍴(glicerina) =", _rho_glycerol := rho_glycerol(T), "kg/m³")
    ln_eta_water = ln(_eta_water)
    ln_eta_glycerol = ln(_eta_glycerol)
    ln_diff = (ln_eta_water - ln_eta_glycerol)
    rho_sol = _rho_glycerol  # kg/m^3
    last_rho_sol_best = rho_sol.best
    for i in count(1):
        print(f"Iterazione {i}:")
        print(" con ⍴ =", rho_sol, "kg/m³")
        eta_sol = (2/9) * (rho_sfere - rho_sol) * g * xi * 10**3   # mPa s
        print(" -> η =", eta_sol, "mPa s")
        alpha = (ln(eta_sol) - ln_eta_glycerol)/ln_diff
        print(" -> ⍺ =", alpha)
        cm = alpha2wg(T, alpha)
        print(f" -> c = {cm*100}% m/s")
        rho_sol = rho_solution(T, cm)
        print(" -> ⍴ =", rho_sol, "kg/m³")
        if abs(rho_sol.best - last_rho_sol_best) <= best_threshold:
            break
        last_rho_sol_best = rho_sol.best
    return eta_sol, cm, rho_sol


In [4]:
# NOTA: T è in °C
# T1_piccole = d(24.8, 0.2)
T1 = d(24.6, 0.2)   # °C
xi1 = d(1.822967460e-5, 6.6444977398e-8)
print("--- Giorno 1 ---")
eta1, c1, rho1 = analysis(T1, xi1, best_threshold=1e-10)

--- Giorno 1 ---
T = 24.6 ± 0.2 °C
ξ = 1.82296746e-05 ± 6.6444977398e-08
η(acqua) = 0.900951687825388 ± 0.00413662225747271 mPa s
η(glicerina) = 844.740864190873 ± 15.9305079009403 mPa s
⍴(acqua) = 996.9907576701406 ± 0.04991080876876254 kg/m³
⍴(glicerina) = 1257.9448 ± 0.12240000000000001 kg/m³
Iterazione 1:
 con ⍴ = 1257.9448 ± 0.12240000000000001 kg/m³
 -> η = 256.4237529735773 ± 2.988594782528412 mPa s
 -> ⍺ = 0.17421312876759912 ± 0.00505581564171701
 -> c = 93.5605863290863 ± 0.212180421620979% m/s
 -> ⍴ = 1241.53018353832 ± 1.02693320165069 kg/m³
Iterazione 2:
 con ⍴ = 1241.53018353832 ± 1.02693320165069 kg/m³
 -> η = 257.075815183165 ± 3.02756855048707 mPa s
 -> ⍺ = 0.17384201123760548 ± 0.00507237764907506
 -> c = 93.5761566997101 ± 0.212752887185449% m/s
 -> ⍴ = 1241.57154122762 ± 1.02918612136684 kg/m³
Iterazione 3:
 con ⍴ = 1241.57154122762 ± 1.02918612136684 kg/m³
 -> η = 257.074172270196 ± 3.02765038292910 mPa s
 -> ⍺ = 0.17384294510887666 ± 0.00507243836301076
 -> c = 93

In [5]:
xi2 = d(2.6374145872e-5, 9.5797126107e-8)
T2 = d(19.4, 0.2)   # °C
print("--- Giorno 2 ---")
eta2, c2, rho2 = analysis(T2, xi2, best_threshold=1e-10)

--- Giorno 2 ---
T = 19.4 ± 0.2 °C
ξ = 2.6374145872e-05 ± 9.5797126107e-08
η(acqua) = 1.01982580168225 ± 0.00504785762682952 mPa s
η(glicerina) = 1398.00458235919 ± 27.8341453877844 mPa s
⍴(acqua) = 998.1691707971398 ± 0.04060593951868824 kg/m³
⍴(glicerina) = 1261.1272 ± 0.12240000000000001 kg/m³
Iterazione 1:
 con ⍴ = 1261.1272 ± 0.12240000000000001 kg/m³
 -> η = 370.8032872620404 ± 4.318267633461683 mPa s
 -> ⍺ = 0.18373229721313855 ± 0.00500100942668852
 -> c = 93.1710194049245 ± 0.212271623636351% m/s
 -> ⍴ = 1243.72560825954 ± 1.01690084800264 kg/m³
Iterazione 2:
 con ⍴ = 1243.72560825954 ± 1.01690084800264 kg/m³
 -> η = 371.803394923951 ± 4.37432908081660 mPa s
 -> ⍺ = 0.1833593989492096 ± 0.00501626405881043
 -> c = 93.1868427704386 ± 0.212796482143604% m/s
 -> ⍴ = 1243.76770258301 ± 1.01900331412257 kg/m³
Iterazione 3:
 con ⍴ = 1243.76770258301 ± 1.01900331412257 kg/m³
 -> η = 371.800975670813 ± 4.37443865974410 mPa s
 -> ⍺ = 0.18336029977658708 ± 0.00501631856024698
 -> c = 93

In [6]:
print(rho2.ε(rho1))
print(c2.ε(c1))

1.07223591805422
-0.914836201309424


In [8]:
def Re(v: Measure[float], rho: Measure[float], eta: Measure[float]) -> Measure[float]:
    return (v * rho * d(8.0, 0.1)*10)/eta

for v in [d(8.738e-2, 0.016e-2), d(12.97e-2, 0.03e-2), d(19.58e-2, 0.05e-2)]:
    print(Re(v, rho1, eta1))

for v in [d(5.65e-2, 0.02e-2), d(8.61e-2, 0.04e-2), d(14.48e-2, 0.06e-2)]:
    print(Re(v, rho2, eta2))

33.7609988608565 ± 0.909432604386998
50.1121715753386 ± 1.37404166851907
75.6512196950756 ± 2.09250638839769
15.1205344714892 ± 0.432820128792665
23.0420888140747 ± 0.685055028581048
38.7513874596750 ± 1.13264420955994
